<a href="https://colab.research.google.com/github/guillerZeva/Bot-Agencia-de-Viajes-Telegram/blob/main/TravelBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalar librerias 

In [ ]:
!pip install python-telegram-bot
!pip install emoji

In [ ]:
import logging
from uuid import uuid4
from telegram import InlineQueryResultArticle, ParseMode, InputTextMessageContent, Update, ReplyKeyboardMarkup
from telegram import InlineKeyboardButton, InlineKeyboardMarkup, ReplyKeyboardRemove,KeyboardButton
from telegram.ext import Updater, InlineQueryHandler, CommandHandler, CallbackContext, CallbackQueryHandler
from telegram.ext import ConversationHandler, MessageHandler,Filters
from telegram.utils.helpers import escape_markdown


from bs4 import BeautifulSoup as bs
import requests
import json
from emoji import emojize

from geopy.geocoders import Nominatim


FECHA1,ES,FECHA2 = range(3)
INPUT,OTRO_LUGAR,TICKET = range(3)

logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)



#------------------------------------------------------------------CLASES----------------------------------------------------------------------------------------------------

class Disponible:
  def __init__(self, lista_destino):
    self.lista_destino = lista_destino 

class Viaje:
  def __init__(self,paisD,costo,origen, iataOrigen, destino, iataDestino, fechaInicial, fechaFinal):
        self.paisD = paisD
        self.costo = costo
        self.origen = origen
        self.iataOrigen = iataOrigen
        self.destino = destino
        self.iataDestino = iataDestino
        self.fechaInicial = fechaInicial
        self.fechaFinal = fechaFinal


#-------------------------------------------------------------BUSCAR VIAJES----------------------------------------------------------------------------------------
def emoji_pais(pais):
  with open('paises.json', encoding='utf-8') as contenido:
    paises = json.load(contenido)
    valor ='' 
    for p in paises:
          if p['pais'] == pais:
            emoji = p['emoji']
            valor = emoji
  return valor

def paises_viaje(viajes):
  paises = []
  for v in viajes:
    viaje = v.destino + " " + emojize(":"+emoji_pais(Buscar_json_pais(v.iataDestino))+":", use_aliases=True)
    paises.append(viaje)
    
  return paises

def Buscar_json_pais(iata):
  with open('IATA.json') as contenido:
    iatas = json.load(contenido)
    valor ='' 
    for pais in iatas:
      for i in iatas[pais]:
        if i['IATA'] == iata:
          valor = pais
  return valor

def contiene(vuelos,destino):
  no = True
  for i in vuelos:
    if i.destino == destino:
      no = False
      break
  return no

def buscar_viaje(ciudad):

  url ='https://www.latam.com/es-ec/vuelos-desde-'+ ciudad.lower()
  res = requests.get(url)
  soup = bs(res.content)
  lugares = soup.find_all('div', class_="card-body")
  vuelos = set ()

  for v in lugares:
    destino = v.find('span', class_='fare-atom-origin-destination-destination-city')
    costo = v.find('span', class_='fare-atom-price-total-price')
    fechas = v.find_all('p', class_='fare-atom-trip-leg-date')
    iataD = v.find('span', class_='fare-atom-origin-destination-destination-airport')

    if ( destino != None and contiene(vuelos,destino.get_text())):
      iata = iataD.get_text().replace(' ','').replace('(','').replace(')','')
      viaje = Viaje('Ecuador',costo.get_text() , ciudad,'GYE', destino.get_text(),iata ,fechas[0].get_text(), fechas[1].get_text())
      vuelos.add(viaje)

  list_viajes = list(vuelos)
  return list_viajes

lista = []
disponible = Disponible(lista)
#-----------------------------------------------------------------------------------------------------------------------------------------------------
#POSICIONAR BOTONES EN COLUMNAS
def build_menu(buttons,n_cols,header_buttons=None,footer_buttons=None):
  menu = [buttons[i:i + n_cols] for i in range(0, len(buttons), n_cols)]
  if header_buttons:
    menu.insert(0, header_buttons)
  if footer_buttons:
    menu.append(footer_buttons)
  return menu

def mostrarBotones(lista,columns,callback):
  button_list = []
  for each in lista:

    button_list.append(InlineKeyboardButton(each, callback_data = each))
    

  reply_markup=InlineKeyboardMarkup(build_menu(button_list,n_cols=columns)) #n_cols = 1 is for single column and mutliple rows
  return reply_markup

#--------------------------------------------------------------------------------BOT---------------------------------------------------------------------
def start(update: Update, context: CallbackContext) -> None:

  update.message.reply_text(f'¡Hola, {update.message.chat.first_name} {emojize(":smile:", use_aliases=True)}!  \n'+
                              f'{emojize(":sparkles:", use_aliases=True)}¡Bienvenido a Travel Bot!{emojize(":sparkles:", use_aliases=True)}\n'+
                              '\nTravel bot te ayudará a buscar y reservar un vuelo'+ 
                              f' desde donde estés hacia cualquier parte del mundo {emojize(":airplane:", use_aliases=True)}{emojize(":earth_americas:", use_aliases=True)}.'+
                              f'\n\n')
  menu(update,context)

def menu(update: Update, context: CallbackContext) -> None:
  emoji_avion = emojize(":airplane:", use_aliases=True)
  emoji_buscar = emojize(":mag_right:", use_aliases=True)
  emoji_cod = emojize(":man_technologist:", use_aliases=True)

  vd = 'Vuelos Disponible'+' '+str(emoji_avion)
  b = 'Buscar'+' '+str(emoji_buscar)
  c = 'Código'+' '+str(emoji_cod)

  text_menu = f'{emojize(":sparkles:", use_aliases=True)} Menú de navegación {emojize(":sparkles:", use_aliases=True)}\n'
  
  botones_menu = [[InlineKeyboardButton(vd, callback_data = 'Vuelos_Disponible' )],
                  [InlineKeyboardButton(b, callback_data = 'buscar')],
                  [InlineKeyboardButton(c, url='https://github.com/guillerZeva/Bot-Agencia-de-Viajes-Telegram')]]

  reply_markup = InlineKeyboardMarkup(botones_menu)
  context.bot.send_message(chat_id=update.message.chat_id, text=text_menu ,reply_markup=reply_markup)


def botones_teclado(update, context):
  main_menu_keyboard = [[KeyboardButton(text= 'Menu',request_location = True)],
                          [KeyboardButton('Mi Agenda')]]
  reply_kb_markup = ReplyKeyboardMarkup(main_menu_keyboard,
                                                  resize_keyboard=True,
                                                  one_time_keyboard=True)

  


def lugar_Vuelos_Disponible(update,context):
  query = update.callback_query
  query.answer()


  main_menu_keyboard = [[KeyboardButton(text= 'Ubicacion Actual',request_location = True)],
                          [KeyboardButton('Otro lugar')]]
  reply_kb_markup = ReplyKeyboardMarkup(main_menu_keyboard,
                                                   resize_keyboard=True,
                                                   one_time_keyboard=True)

    # Send the message with menu
  context.bot.send_message(chat_id=query.message.chat_id,
                     text='f',
                     reply_markup=reply_kb_markup) 
  query.edit_message_text(text='ESCOGER lugar')

  return INPUT

def location(update,context):

  punto = str(update.message.location.latitude)+','+str(update.message.location.longitude)

  geolocalizador = Nominatim()
  direccion = geolocalizador.reverse(punto)
  aux = direccion.address.split(', ')
    
  print(aux[3])

  disponible.lista_destino = buscar_viaje(aux[3]) 
  list_of_cities = paises_viaje(disponible.lista_destino)
  reply_markup = mostrarBotones(list_of_cities,1,'viajes')
  update.message.reply_text(text=f'Viajes Disponibles desde {aux[3]}: ', reply_markup=reply_markup)
 
  return TICKET


#6° Pasaje 38A NO, Bastión Popular, Tarqui, Guayaquil, 090704, Ecuador

def ingrese_otroLugar(update,context):
  update.message.reply_text('¡Hola,' )

  return OTRO_LUGAR

def input_lugar(update,context):
  
  disponible.lista_destino = buscar_viaje(update.message.text) 
  list_of_cities = paises_viaje(disponible.lista_destino)
  reply_markup = mostrarBotones(list_of_cities,1,'viajes')
  update.message.reply_text(text=f'Viajes Disponibles desde: {update.message.text}', reply_markup=reply_markup)
 
  return TICKET

def ticket(update,context):
  
  query = update.callback_query
  query.answer()

  list_of_cities = paises_viaje(disponible.lista_destino)

  for lugar in list_of_cities:
    if update.callback_query.data == lugar:

      emoji_ida = emojize(":ticket:", use_aliases=True)
      ti = emoji_ida + ' ' + 'Ticket ida'
      tiv = emoji_ida + ' ' + 'Ticket ida y vuelta'

      botones_menu = [[InlineKeyboardButton(ti, callback_data = 'ticket_ida' )],
                  [InlineKeyboardButton(tiv, callback_data = 'ticket_ida_vuelta')]]

      reply_markup = InlineKeyboardMarkup(botones_menu)
      query.edit_message_text(text=lugar, reply_markup=reply_markup)
 



def press_button_callback(update, context): 
  query = update.callback_query
  query.answer()


  list_of_cities =[]
  for lugar in list_of_cities:
    if update.callback_query.data == lugar:

      pass



def calendar_handler2(update, context):
    query = update.callback_query
    query.answer()
    
    query.edit_message_text(
        text = "La Fecha de regreso: ",
        reply_markup=create_calendar()                   
    )

    return ES

def calendar_handler1(update, context):
    update.message.reply_text("Por favor escoger una fecha: ",
                        reply_markup=create_calendar())
  
    return FECHA1

def inline_handler(update, context):
    selected,date = process_calendar_selection(context.bot, update)
    if selected:

        reply_markup= InlineKeyboardMarkup([[InlineKeyboardButton(text="Back", callback_data="help_back")]])

        context.bot.send_message(chat_id=update.callback_query.from_user.id,
                        text="La fecha de partida es: %s" % (date.strftime("%d/%m/%Y")))
        
        context.bot.send_message(chat_id=update.callback_query.from_user.id,
                        text="Regreso",
                        reply_markup=reply_markup)
      

    
    return FECHA2

def inline_handler2(update, context):
    selected,date = process_calendar_selection(context.bot, update)
    if selected:
        context.bot.send_message(chat_id=update.callback_query.from_user.id,
                        text="La fecha de regreso es: %s" % (date.strftime("%d/%m/%Y")),
                        reply_markup=ReplyKeyboardRemove())



def inlinequery(update: Update, context: CallbackContext) -> None:
    """Handle the inline query."""
    query = update.inline_query.query
    results = [

        InlineQueryResultArticle(
            id=uuid4(),
            title="Bold",
            input_message_content=InputTextMessageContent(
                f"*{escape_markdown(query)}*", parse_mode=ParseMode.MARKDOWN
            ),
        ),
        InlineQueryResultArticle(
            id=uuid4(),
            title="PERDO",
            input_message_content=InputTextMessageContent(
                f"*{escape_markdown(query)}*", parse_mode=ParseMode.MARKDOWN
            ),
        ) ,
        InlineQueryResultArticle(
            id=uuid4(),
            title="PUTO",
            input_message_content=InputTextMessageContent(
                f"*{escape_markdown(query)}*", parse_mode=ParseMode.MARKDOWN
            ),
        )  ,
        InlineQueryResultArticle(
            id=uuid4(),
            title="ALGO",
            input_message_content=InputTextMessageContent(
                f"*{escape_markdown(query)}*", parse_mode=ParseMode.MARKDOWN
            ),
        )   
    ]

    update.inline_query.answer(results)



def main() -> None:

  updater = Updater("TOKEN")

  dispatcher = updater.dispatcher

  dispatcher.add_handler(CommandHandler("start", start))


  vuelos_handler = ConversationHandler(
  entry_points=[CallbackQueryHandler(callback= lugar_Vuelos_Disponible, pattern='Vuelos_Disponible')],
  states={
      INPUT: [MessageHandler(Filters.text, ingrese_otroLugar),MessageHandler(Filters.location, location)],
      OTRO_LUGAR:  [MessageHandler(Filters.text, input_lugar)],
      TICKET: [CallbackQueryHandler(callback= ticket)]
  },
  fallbacks=[]
  )

  dispatcher.add_handler(vuelos_handler)










    
  dispatcher.add_handler(CallbackQueryHandler(callback = press_button_callback))

  dispatcher.add_handler(InlineQueryHandler(inlinequery))

  conv_handler = ConversationHandler(
  entry_points=[CommandHandler("calendar",calendar_handler1)],
  states={
        FECHA1: [CallbackQueryHandler(inline_handler)],
        FECHA2: [CallbackQueryHandler(calendar_handler2, pattern='help_back')],
        ES:     [CallbackQueryHandler(inline_handler2)],

  },
  fallbacks=[CommandHandler("calendar",calendar_handler1)]
  )

  dispatcher.add_handler(conv_handler)

  updater.start_polling()

  updater.idle()

if __name__ == '__main__':
  
  main()




/usr/local/lib/python3.7/dist-packages/telegram/ext/conversationhandler.py:280: UserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message.
  "If 'per_message=False', 'CallbackQueryHandler' will not be "
2021-03-02 05:37:24,708 - apscheduler.scheduler - INFO - Scheduler started
2021-03-02 05:37:50,897 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2021-03-02 05:37:50,900 - apscheduler.scheduler - INFO - Scheduler has been shut down


#CALENDARIO 

In [ ]:
from telegram import InlineKeyboardButton, InlineKeyboardMarkup,ReplyKeyboardRemove
import datetime
import calendar

def create_callback_data(action,year,month,day):
    return ";".join([action,str(year),str(month),str(day)])

def separate_callback_data(data):
    return data.split(";")


def create_calendar(year=None,month=None):

    now = datetime.datetime.now()
    if year == None: year = now.year
    if month == None: month = now.month
    data_ignore = create_callback_data("IGNORE", year, month, 0)
    keyboard = []
    #First row - Month and Year
    row=[]
    row.append(InlineKeyboardButton(calendar.month_name[month]+" "+str(year),callback_data=data_ignore))
    keyboard.append(row)
    #Second row - Week Days
    row=[]
    for day in ["Mo","Tu","We","Th","Fr","Sa","Su"]:
        row.append(InlineKeyboardButton(day,callback_data=data_ignore))
    keyboard.append(row)

    my_calendar = calendar.monthcalendar(year, month)
    for week in my_calendar:
        row=[]
        for day in week:
            if(day==0):
                row.append(InlineKeyboardButton(" ",callback_data=data_ignore))
            else:
                row.append(InlineKeyboardButton(str(day),callback_data=create_callback_data("DAY",year,month,day)))
        keyboard.append(row)
    #Last row - Buttons
    row=[]
    row.append(InlineKeyboardButton("<",callback_data=create_callback_data("PREV-MONTH",year,month,day)))
    row.append(InlineKeyboardButton(" ",callback_data=data_ignore))
    row.append(InlineKeyboardButton(">",callback_data=create_callback_data("NEXT-MONTH",year,month,day)))
    keyboard.append(row)

    return InlineKeyboardMarkup(keyboard)


def process_calendar_selection(bot,update):

    ret_data = (False,None)
    query = update.callback_query
    (action,year,month,day) = separate_callback_data(query.data)
    curr = datetime.datetime(int(year), int(month), 1)
    if action == "IGNORE":
        bot.answer_callback_query(callback_query_id= query.id)
    elif action == "DAY":
        bot.edit_message_text(text=query.message.text,
            chat_id=query.message.chat_id,
            message_id=query.message.message_id
            )
        ret_data = True,datetime.datetime(int(year),int(month),int(day))
    elif action == "PREV-MONTH":
        pre = curr - datetime.timedelta(days=1)
        bot.edit_message_text(text=query.message.text,
            chat_id=query.message.chat_id,
            message_id=query.message.message_id,
            reply_markup=create_calendar(int(pre.year),int(pre.month)))
    elif action == "NEXT-MONTH":
        ne = curr + datetime.timedelta(days=31)
        bot.edit_message_text(text=query.message.text,
            chat_id=query.message.chat_id,
            message_id=query.message.message_id,
            reply_markup=create_calendar(int(ne.year),int(ne.month)))
    else:
        bot.answer_callback_query(callback_query_id= query.id,text="Algo salió mal!")
        # UNKNOWN
    return ret_data